In [ ]:
# Boilerplate
import sys
import numpy as np
import tensorflow as tf
import matplotlib
from matplotlib.pyplot import figure, imshow, gca, tight_layout, show
from matplotlib import patches
from importlib import reload
import os

if 'roi_pooling' not in sys.modules:
    import backbone
    import classifier
    import data_utils
    import faster_rcnn
    import rpn
    import roi_pooling
    import geometry
    import evaluation
else:
    reload(backbone)
    reload(classifier)
    reload(data_utils)
    reload(faster_rcnn)
    reload(rpn)
    reload(roi_pooling)
    reload(geometry)
    reload(evaluation)

In [ ]:
# List physical devices
is_colab = len(tf.config.list_physical_devices('GPU')) > 0

# Data locations
if is_colab:
    datapath='/content'
    backbone_weights='drive/MyDrive/trained_backbone.ckpt'
    rpn_weights='drive/MyDrive/trained_rpn.ckpt'
    class_weights='drive/MyDrive/blank_classifier.ckpt'
else:
    datapath='tensorflow-great-barrier-reef'
    backbone_weights='trained_backbone.ckpt'
    rpn_weights='trained_rpn.ckpt'
    class_weights='blank_classifier.ckpt'

In [ ]:
# Instantiate the high-level wrapper
frcnn = faster_rcnn.FasterRCNNWrapper(
    input_shape=(720, 1280, 3),
    datapath=datapath,
    backbone_type='ResNet50',
    backbone_weights='imagenet', #backbone_weights if os.path.exists(backbone_weights) else 'finetune',
    rpn_weights=rpn_weights if os.path.exists(rpn_weights) else None,
    classifier_weights=class_weights if os.path.exists(class_weights) else None
)

In [ ]:
#images, labels = frcnn.data_loader_full.get_training().__iter__().next()
#_ = frcnn.predict(images, return_mode='dict')
#frcnn.classmodel.save_classifier_state(class_weights)

In [ ]:
# Let's instantiate the classifier
training = frcnn.data_loader_full.get_training().__iter__()

In [ ]:
train_images, train_labels = training.next()
print(train_images.shape)
print(train_labels)

In [ ]:
# Run system in forward mode. First show all the RoI to test.
# This is returned in image space for simplicity
roi_unpool = frcnn.rpnwrapper.propose_regions(train_images, input_images=True, output_images=True)

# Next show the pooled RoI. Note that we have to do this in feature space
# because that is the language the pooling class expects.
features=frcnn.backbone.extractor(train_images) 
regions = frcnn.rpnwrapper.propose_regions(features, input_images=False, output_images=False)
_, roi_pool = frcnn.RoI_pool((features, regions))

# Convert these pooled RoI back to image space for plotting and diagnostic purposes
roi_numpy= roi_pool.numpy().astype('float32')
roi_numpy[:,:,1], roi_numpy[:,:,0]  = frcnn.backbone.feature_coords_to_image_coords(
    roi_numpy[:,:,1], roi_numpy[:,:,0]
)
roi_numpy[:,:,3], roi_numpy[:,:,2]  = frcnn.backbone.feature_coords_to_image_coords(
    roi_numpy[:,:,3], roi_numpy[:,:,2]
)
roi_pool_image = tf.convert_to_tensor(roi_numpy)

# Convert the labels to tensor
all_decoded = [frcnn.data_loader_full.decode_label(label) for label in train_labels]
all_decoded = tf.convert_to_tensor(all_decoded)

In [ ]:
# Plot everything up
for i in range(all_decoded.shape[0]):

    figure(figsize=(16, 9))
    imshow(train_images[i, :, :, :].numpy() / 255.0)

    # Draw the ground truth
    for annotation in all_decoded[i]:
        rect = patches.Rectangle(
            (annotation[0], annotation[1]),
            annotation[2],
            annotation[3],
            linewidth=4,
            edgecolor="y",
            facecolor="none",
        )
        gca().add_patch(rect)

    # Draw the RPN outputs
    for j in range(roi_unpool.shape[1]):
        rect = patches.Rectangle(
            (roi_unpool[i, j, 0], roi_unpool[i, j, 1]),
            roi_unpool[i, j, 2],
            roi_unpool[i, j, 3],
            linewidth=((roi_unpool.shape[1] -j) / 32)+1,
            edgecolor="g",
            facecolor="none",
            linestyle='--'
        )
        gca().add_patch(rect)
    
    # Draw the IoU suppressed and pooled areas
    for j in range(roi_pool.shape[1]):
        rect = patches.Rectangle(
            (roi_pool_image[i, j, 0], roi_pool_image[i, j, 1]),
            roi_pool_image[i, j, 2],
            roi_pool_image[i, j, 3],
            linewidth=((roi_pool.shape[1] - j) /4) + 1,
            edgecolor="r",
            facecolor="none",
            linestyle=':'
        )
        gca().add_patch(rect)   
    
    # Plot it up
    gca().grid("True")
    tight_layout()

In [ ]:
# Compile to enable training mode
frcnn.classmodel.compile(
    optimizer=frcnn.optimizer, metrics=frcnn.validation_f2s
)

print(frcnn.classmodel)
print(frcnn.optimizer)

In [ ]:
# Sure enough the loss looks weird in test_step, zero overlap and loss=16
frcnn.classmodel.test_step((train_images, train_labels))

In [ ]:
# Same with the train_step
frcnn.classmodel.train_step((train_images, train_labels))

In [ ]:
# Now let's run the basic call method
features_unpool = frcnn.backbone(train_images)
features, roi = frcnn.RoI_pool(
                (
                    features_unpool,
                    frcnn.rpnwrapper.propose_regions(
                        features_unpool, input_images=False, output_images=False
                    ),
                )
            )

print(features.shape)
print(roi)

In [ ]:
for i in range(10):
    print(np.max(features_unpool[3,2:5,2:5,:]))
    print(np.max(features[3,i,:,:,:]))

In [ ]:
outputs = frcnn.classmodel.call((features, roi))
print('x')
print(outputs[:,:,0])
print('y')
print(outputs[:,:,1])
print('w')
print(outputs[:,:,2])
print('h')
print(outputs[:,:,3])
print('score')
print(outputs[:,:,4])

In [ ]:
# Ok, so now let's pass through the loss calculation *manually*
cls, bbox = frcnn.classmodel.classifier(features, training=False)
print(cls)

In [ ]:
def _compute_loss(data):
    """
    Compute the loss term for the full network.
    Works on one image at a time.

    Arguments:

    data : (tf.tensor, tf.tensor, tf.tensor, tf.tensor)
        Packed classifier scores, bbox regressors, roi, and labels for this image.
        Note that the RoI should be in *feature coordinates*, not image coordinates.

    """
    
    print("Python interpreter in classifier._compute_loss()")

    # No batch dimensions in this function, called with map_fn
    cls, bbox, roi, labels = data
    
    # Conver to image coordinates
    roi = tf.cast(roi, tf.float32)
    x, y = frcnn.backbone.feature_coords_to_image_coords(roi[:, 0], roi[:, 1])
    w, h = frcnn.backbone.feature_coords_to_image_coords(roi[:, 2], roi[:, 3])
    roi = tf.stack([x, y, w, h], axis=0)
    
    def _calc_IoU(sf):
        return geometry.calculate_IoU(sf, roi) 
    
    # Build a (nstarfish, nroi) tensor of all the IoU values
    IoUs = tf.map_fn(_calc_IoU, labels)
    
    # For each starfish, grab the highest IoU roi or set to -1 if the IoUs are all zero
    # Returns [n_roi,] tensor containing the index of the starfish if it is a + match
    match = tf.where(tf.math.count_nonzero(IoUs, axis=0) > 0, tf.math.argmax(IoUs, axis=0), -1)
    
    # First the regularization term, turned down to match what's in the RPN
    # This regularization is on the outputs of the classifier network, not weights
    # which is done implicitly by the SGDW optimizer
    print('Regularization loss')
    loss = tf.nn.l2_loss(cls) / (100.0 * tf.size(cls, out_type=tf.float32))
    print(loss)
    loss += tf.nn.l2_loss(bbox) / (10.0 * tf.size(bbox, out_type=tf.float32))
    print(loss)
    
    for i in tf.range(frcnn.classmodel.n_proposals, dtype=tf.int64):

        # Classification score
        cls_select = tf.nn.softmax(cls[i :: frcnn.classmodel.n_proposals])

        # Found a real starfish
        if match[i] > 0:
            print('Positive match')
            truth_box = labels[match[i],:]
            t_x_star = (truth_box[0] - roi[0,i]) / roi[2,i]
            t_y_star = (truth_box[1] - roi[1,i]) / roi[3,i]
            t_w_star = geometry.safe_log(truth_box[2] / roi[2,i])
            t_h_star = geometry.safe_log(truth_box[3] / roi[3,i])
            loss += frcnn.classmodel.bbox_reg_l1(
            [t_x_star, t_y_star, t_w_star, t_h_star],
            bbox[i :: frcnn.classmodel.n_proposals],
            )
            print('bounding box loss')
            print(loss)
            loss += frcnn.classmodel.class_loss(cls_select, tf.constant([0.0, 1.0]))
            print('classification loss')
            print(loss)
        else:
            print('negative match')
            loss += frcnn.classmodel.negative_weight * frcnn.classmodel.class_loss(cls_select, tf.constant([1.0, 0.0]))
            print('classification loss')
            print(loss)
            
    return loss

In [ ]:
loss = tf.reduce_sum(
    tf.map_fn(
        _compute_loss,
        [cls, bbox, roi, frcnn.data_loader_full.decode_label(train_labels)],
        fn_output_signature=(tf.float32),
        )
)
print(loss)